In [ ]:
# Run some setup code for this notebook.

import matplotlib.pyplot as plt


# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

## Load Dataset

In [ ]:
import pandas as pd
df = pd.read_csv('loan.csv')
df.head()

## Dataset Info

In [ ]:
df.info()

## Data Cleaning

In [ ]:
df.drop("Loan_ID", axis=1, inplace=True)
# Checking the Missing Values
df.isnull().sum()

In [ ]:
################################################################################
# TODO:                                                                        #
# Task1 deal with NULL rows, you can either choose to drop them or replace them with mean or other value #
################################################################################

# fill numeric values with mean
mean = df.mean(numeric_only=True)
df = df.fillna(value=mean)

# fill string values with mode
mode = df.mode()
mode = { k:mode[k][0] for k in mode }
df = df.fillna(value=mode)

df.isnull().sum()

################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

## Encode 

In [ ]:
################################################################################
# TODO:                                                                        #
# Task2 deal with categorical features
# Tip use pd.get_dummies. 
################################################################################

df = pd.get_dummies(df)

# drop useless columns for 2-value properties
df = df.drop(["Gender_Female", "Married_No", "Education_Not Graduate", "Self_Employed_No", "Loan_Status_N"], axis=1)

df.head()

################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

## Data process

In [ ]:
################################################################################
# TODO:                                                                        #
# Task3 split the dataset into X_train, X_test, y_train, y_test
# Optional: you can also use normalization
################################################################################
df.sample(frac=1).reset_index()
train_size = int(df.shape[0] * 0.8)

train, test = df.loc[0:train_size], df.loc[train_size:]

train_X = train.drop(["Loan_Status_Y"], axis=1)
train_Y = train["Loan_Status_Y"]
test_X = test.drop(["Loan_Status_Y"], axis=1)
test_Y = test["Loan_Status_Y"]

train_X.info()

################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

## Train

In [ ]:
from Logistic import LogisticRegression
import matplotlib.pyplot as plt

################################################################################
# TODO:                                                                        #
# Task4 train your model and plot the loss curve of training
# You need to complete the Logistic.py file
# model = LogisticRegression()
################################################################################


################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

## Test

In [ ]:
################################################################################
# TODO:                                                                        #
# Task5 compare the accuracy(or other metrics you want) of test data with different parameters you train with
################################################################################




################################################################################
#                                 END OF YOUR CODE                             #
################################################################################